In [7]:
import os
import json
import math
from datetime import date, time
import numpy as np
import psycopg2
import pandas as pd


## MIGRACION DATOS A POSTGRESQL YELLOW DATA

In [2]:
# Establecer la semilla aleatoria para reproducibilidad
np.random.seed(123)

In [3]:
# Obtener la lista de archivos Parquet procesados en el directorio
directorio = "../datasets/processed/yellow_analytics/yellow_analytics_part_1_1.parquet"

# Lee el archivo Parquet utilizando Dask
df = pd.read_parquet(directorio)
#df_dask_particionado = df_dask.repartition(npartitions=4)
print(df.shape)
df.head()

(363419, 11)


,IdProveedor,TotalPasajeros,DistanciaViaje,IdTipoTarifa,IdZonaOrigen,IdZonaDestino,IdTipoPago,CostoTotal,FechaRecogida,HoraRecogida,DuracionViaje
0,1,2,3.80,1.0,142,236,1,21.95,2022-01-01,0,1069
1,1,1,2.10,1.0,236,42,1,13.30,2022-01-01,0,504
3,2,1,1.09,1.0,114,68,2,11.80,2022-01-01,0,602
4,2,1,4.30,1.0,68,163,1,30.30,2022-01-01,0,2252
6,2,1,5.07,1.0,233,87,1,26.00,2022-01-01,0,848


In [4]:
df.shape

(363419, 11)

In [5]:
df.IdProveedor.unique()

array([1, 2], dtype=int64)

In [6]:
df.IdProveedor.value_counts()

IdProveedor
2    256545
1    106874
Name: count, dtype: int64

In [8]:
def estratificar_muestra(df, columna, proporcion):
    """
    Realiza muestreo estratificado en un DataFrame según una columna específica.

    Parameters:
    - df (pd.DataFrame): El DataFrame de origen.
    - columna (str): El nombre de la columna sobre la que se realizará el muestreo estratificado.
    - proporcion (float): La proporción deseada de la muestra respecto al tamaño del DataFrame original. Por defecto, 0.12.

    Returns:
    - pd.DataFrame: DataFrame con la muestra estratificada.
    """
    # Calcular el tamaño de muestra proporcional para cada categoría en la columna especificada
    proporciones = df[columna].value_counts(normalize=True) * proporcion
    tamaño_muestra_por_categoria = (proporciones * len(df)).astype(int)

    # Lista para almacenar las muestras aleatorias estratificadas
    muestras_estratificadas = []

    # Iterar sobre cada categoría única en la columna especificada
    for categoria in df[columna].unique():
        # Seleccionar aleatoriamente el tamaño de muestra proporcional para la categoría actual
        muestra_categoria = df[df[columna] == categoria].sample(tamaño_muestra_por_categoria[categoria])
        # Agregar las muestras seleccionadas al DataFrame de muestras estratificadas
        muestras_estratificadas.append(muestra_categoria)

    # Concatenar las muestras estratificadas en un solo DataFrame
    muestras_estratificadas = pd.concat(muestras_estratificadas)

    return muestras_estratificadas

# Ejemplo de uso:
# Reemplaza 'df' con tu DataFrame y 'columna' con el nombre de la columna sobre la que deseas realizar el muestreo estratificado
muestra_estratificada = estratificar_muestra(df, columna='IdProveedor', proporcion=0.12)

In [9]:
muestra_estratificada.shape

(43609, 11)

In [10]:
muestra_estratificada.IdProveedor.value_counts()

IdProveedor
2    30785
1    12824
Name: count, dtype: int64

In [11]:
def procesar_archivos_parquet(ruta_directorio):
    """
    Función para procesar archivos Parquet en un directorio.

    Args:
    ruta_directorio (str): La ruta del directorio que contiene los archivos Parquet.

    Returns:
    list: Lista de nombres de archivos Parquet en el directorio.
    """
    archivos_parquet = []

    # Obtener la lista de nombres de archivos Parquet en el directorio
    archivos = os.listdir(ruta_directorio)

    # Filtrar los nombres de los archivos Parquet
    for archivo in archivos:
        if archivo.endswith('.parquet'):
            archivos_parquet.append(archivo)

    return archivos_parquet

ruta_directorio = r'C:\Users\ozi\ti\MVP_ML_TransporteNY\datasets\processed\yellow_analytics'
files = procesar_archivos_parquet(ruta_directorio)

In [16]:
ruta_directorio = r'C:\Users\ozi\ti\MVP_ML_TransporteNY\datasets\processed\yellow_analytics'
files = procesar_archivos_parquet(ruta_directorio)

In [17]:
files

['yellow_analytics_part_10_1.parquet',
 'yellow_analytics_part_10_2.parquet',
 'yellow_analytics_part_10_3.parquet',
 'yellow_analytics_part_10_4.parquet',
 'yellow_analytics_part_11_1.parquet',
 'yellow_analytics_part_11_2.parquet',
 'yellow_analytics_part_11_3.parquet',
 'yellow_analytics_part_11_4.parquet',
 'yellow_analytics_part_12_1.parquet',
 'yellow_analytics_part_12_2.parquet',
 'yellow_analytics_part_12_3.parquet',
 'yellow_analytics_part_12_4.parquet',
 'yellow_analytics_part_13_1.parquet',
 'yellow_analytics_part_13_2.parquet',
 'yellow_analytics_part_13_3.parquet',
 'yellow_analytics_part_13_4.parquet',
 'yellow_analytics_part_14_1.parquet',
 'yellow_analytics_part_14_2.parquet',
 'yellow_analytics_part_14_3.parquet',
 'yellow_analytics_part_14_4.parquet',
 'yellow_analytics_part_15_1.parquet',
 'yellow_analytics_part_15_2.parquet',
 'yellow_analytics_part_15_3.parquet',
 'yellow_analytics_part_15_4.parquet',
 'yellow_analytics_part_16_1.parquet',
 'yellow_analytics_part_1

In [ ]:
def procesar_archivos_parquet(ruta_directorio):
    """
    Función para procesar archivos Parquet en un directorio.

    Args:
    ruta_directorio (str): La ruta del directorio que contiene los archivos Parquet.

    Returns:
    list: Lista de nombres de archivos Parquet en el directorio.
    """
    archivos_parquet = []

    # Obtener la lista de nombres de archivos Parquet en el directorio
    archivos = os.listdir(ruta_directorio)

    # Filtrar los nombres de los archivos Parquet
    for archivo in archivos:
        if archivo.endswith('.parquet'):
            archivos_parquet.append(archivo)

    return archivos_parquet

def procesar_archivos_parquet(ruta_directorio):
    """
    Función para procesar archivos Parquet en un directorio.

    Args:
    ruta_directorio (str): La ruta del directorio que contiene los archivos Parquet.

    Returns:
    list: Lista de nombres de archivos Parquet en el directorio.
    """
    archivos_parquet = []

    # Obtener la lista de nombres de archivos Parquet en el directorio
    archivos = os.listdir(ruta_directorio)

    # Filtrar los nombres de los archivos Parquet
    for archivo in archivos:
        if archivo.endswith('.parquet'):
            archivos_parquet.append(archivo)

    return archivos_parquet

ruta_directorio = r'C:\Users\ozi\ti\MVP_ML_TransporteNY\datasets\processed\yellow_analytics'
files = procesar_archivos_parquet(ruta_directorio)

def cargar_parquet_a_blob(archivo_parquet, ruta_directorio, account_name, account_key, container_name):
    """
    Función para cargar un archivo Parquet en Azure Blob Storage.

    Args:
    archivo_parquet (str): Nombre del archivo Parquet que se desea cargar.
    ruta_directorio (str): La ruta del directorio que contiene el archivo Parquet.
    account_name (str): El nombre de la cuenta de almacenamiento.
    account_key (str): La clave de la cuenta de almacenamiento.
    container_name (str): El nombre del contenedor Blob.

    Returns:
    None
    """
    # Generar la ruta completa del archivo Parquet
    ruta_completa = os.path.join(ruta_directorio, archivo_parquet)

    # Generar la ruta de destino en el contenedor Blob
    #ruta_destino_blob = f"raw-data/yellow_analytics/{archivo_parquet}"
    ruta_destino_blob = f"raw-data/yellow_analytics/{archivo_parquet}"
    # Conexión a la cuenta de almacenamiento
    blob_service_client = BlobServiceClient(account_url=f"https://{account_name}.blob.core.windows.net", credential=account_key)

    # Verificar si el archivo ya existe en el contenedor Blob
    container_client = blob_service_client.get_container_client(container_name)
    blob_client = container_client.get_blob_client(ruta_destino_blob)

    try:
        blob_properties = blob_client.get_blob_properties()
        print(f"El archivo {archivo_parquet} ya existe en el contenedor de Azure Blob Storage. No se volverá a cargar.")
    except Exception as ex:
        # Si el blob no existe, cargar el archivo
        df = pd.read_parquet(ruta_completa)
        
        # Subir archivo al contenedor
        with BytesIO() as bytes_io:
            df.to_parquet(bytes_io)
            blob_client.upload_blob(bytes_io.getvalue(), overwrite=True)
        
        print(f"Archivo {archivo_parquet} cargado exitosamente en Azure Blob Storage.")


def cargar_datos_parquet(ruta_directorio, account_name, account_key, container_name):
    """
    Función para procesar y cargar archivos Parquet en Azure Blob Storage,
    y guardar un registro de tiempo en un archivo de texto.

    Args:
    ruta_directorio (str): La ruta del directorio que contiene los archivos Parquet.
    account_name (str): El nombre de la cuenta de almacenamiento.
    account_key (str): La clave de la cuenta de almacenamiento.
    container_name (str): El nombre del contenedor Blob.

    Returns:
    None
    """
    try:
        # Registro de tiempo de inicio
        inicio = time.time()

        # Obtener la lista de nombres de archivos Parquet en el directorio
        archivos_parquet = procesar_archivos_parquet(ruta_directorio)

        # Iterar sobre cada archivo Parquet
        for archivo_parquet in archivos_parquet:
            # Cargar el archivo Parquet en Azure Blob Storage
            cargar_parquet_a_blob(archivo_parquet, ruta_directorio, account_name, account_key, container_name)
            # Esperar 10 segundos antes de procesar el siguiente archivo
            time.sleep(10)

            # Registro de tiempo de finalización
            fin = time.time()

            # Calcular la duración total de la ejecución
            duracion = fin - inicio

            # Crear la carpeta logs y controles si no existe
            carpeta_controles = os.path.join("logs", "controles")
            if not os.path.exists(carpeta_controles):
                os.makedirs(carpeta_controles)

            # Guardar el registro en un archivo de texto
            with open(os.path.join(carpeta_controles, "blob_logs.txt"), "w") as f:
                f.write(f"Tiempo de inicio: {inicio}\n")
                f.write(f"Tiempo de finalización: {fin}\n")
                f.write(f"Duración total (segundos): {duracion}\n")

    except Exception as e:
        print(f"Error al cargar los datos Parquet: {e}")


# Ejemplo de uso de la función
#ruta_directorio = r'C:\Users\ozi\ti\MVP_ML_TransporteNY\datasets\processed\yellow_analytics'
ruta_directorio = r'C:\Users\ozi\ti\MVP_ML_TransporteNY\datasets\processed\yellow_analytics'
account_name = os.getenv("ACCOUNT_NAME")
account_key = os.getenv("ACCOUNT_KEY")
container_name = 'getdatalakefiles'

# Llamar a la función para procesar y cargar archivos Parquet en Azure Blob Storage
cargar_datos_parquet(ruta_directorio, account_name, account_key, container_name)


In [11]:
muestras_estratificadas.head()

,IdProveedor,TotalPasajeros,DistanciaViaje,IdTipoTarifa,IdZonaOrigen,IdZonaDestino,IdTipoPago,CostoTotal,FechaRecogida,HoraRecogida,DuracionViaje
157178,1,1,2.1,1.0,238,48,1,14.55,2022-01-05,8,694
166028,1,1,2.5,1.0,68,229,1,17.75,2022-01-05,12,928
262756,1,1,1.0,1.0,107,164,1,12.35,2022-01-07,16,335
317951,1,3,2.3,1.0,161,246,1,14.80,2022-01-08,18,933
289658,1,2,5.5,1.0,114,238,1,27.35,2022-01-08,2,1247


In [12]:
muestras_estratificadas.dtypes


IdProveedor         int64
TotalPasajeros      int64
DistanciaViaje    float64
IdTipoTarifa      float64
IdZonaOrigen        int64
IdZonaDestino       int64
IdTipoPago          int64
CostoTotal        float64
FechaRecogida      object
HoraRecogida        int64
DuracionViaje       int64
dtype: object

In [29]:
# Conectar a la base de datos PostgreSQL
conn = psycopg2.connect(
    dbname="nycdb",
    user="postgres",
    password="admin",
    host="localhost"
)

In [30]:
# Crear un cursor para ejecutar comandos SQL
cur = conn.cursor()

In [31]:
muestras_estratificadas.columns

Index(['IdProveedor', 'TotalPasajeros', 'DistanciaViaje', 'IdTipoTarifa',
       'IdZonaOrigen', 'IdZonaDestino', 'IdTipoPago', 'CostoTotal',
       'FechaRecogida', 'HoraRecogida', 'DuracionViaje'],
      dtype='object')

In [16]:
muestras_estratificadas.dtypes

IdProveedor         int64
TotalPasajeros      int64
DistanciaViaje    float64
IdTipoTarifa      float64
IdZonaOrigen        int64
IdZonaDestino       int64
IdTipoPago          int64
CostoTotal        float64
FechaRecogida      object
HoraRecogida        int64
DuracionViaje       int64
dtype: object

In [32]:
# # Crear la tabla en la base de datos
# cur.execute("""
#     CREATE TABLE service_yellow (
#         Id SERIAL PRIMARY KEY,
#         IdProveedor INTEGER,
#         TotalPasajeros INTEGER,
#         DistanciaViaje FLOAT,
#         IdTipoTarifa FLOAT,
#         IdZonaOrigen INTEGER,
#         IdZonaDestino INTEGER,
#         IdTipoPago INTEGER,
#         CostoTotal FLOAT,
#         FechaRecogida DATE,
#         HoraRecogida INTEGER,
#         DuracionViaje INTEGER
#     )
# """)

# # Guardar los cambios en la base de datos
# conn.commit()


# Convertir los valores del DataFrame a listas de Python
registros = muestras_estratificadas.values.tolist()

# Insertar los registros en la tabla
cur.executemany("""
    INSERT INTO service_yellow (
        IdProveedor, TotalPasajeros, DistanciaViaje, IdTipoTarifa, IdZonaOrigen, IdZonaDestino, IdTipoPago, CostoTotal, FechaRecogida, HoraRecogida, DuracionViaje) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
""", registros)

# Guardar los cambios en la base de datos
conn.commit()

# Cerrar la conexión con la base de datos
cur.close()
conn.close()

print("Registros migrados exitosamente a PostgreSQL.")

Registros migrados exitosamente a PostgreSQL.


## MIGRACION DATOS A POSTGRESQL FHVHV DATA

In [1]:
import os
import json
import math
from datetime import date, time
import numpy as np
import psycopg2
import pandas as pd
# Establecer la semilla aleatoria para reproducibilidad
np.random.seed(123)

In [18]:
# Obtener la lista de archivos Parquet procesados en el directorio
directorio = "../datasets/processed/ffvh_analytics/ffvh_analytics_part_1_1.parquet"

# Lee el archivo Parquet utilizando Dask
df = pd.read_parquet(directorio)
#df_dask_particionado = df_dask.repartition(npartitions=4)
print(df.shape)
df.head()

(2845525, 17)


,IdApp,IdProveedor,IdZonaOrigen,IdZonaDestino,DistanciaViaje,trip_time,TarifaPasajero,CostoTotal,FechaRecogida,HoraRecogida,FechaLlegada,HoraLlegada,FechaSolicitada,HoraSolicitada,FechaAtendida,HoraAtendida,DuracionAtencion
0,HV0003,B03404,17,225,1.89,611,9.85,19.18,2022-02-01,00:02,2022-02-01,00:13,2022-01-31,23:58,2022-02-01,00:02,258.0
1,HV0003,B03404,37,37,1.52,493,9.08,16.92,2022-02-01,00:20,2022-02-01,00:28,2022-02-01,00:16,2022-02-01,00:19,187.0
2,HV0003,B03404,37,37,1.50,456,11.75,19.57,2022-02-01,00:39,2022-02-01,00:46,2022-02-01,00:36,2022-02-01,00:38,145.0
3,HV0003,B03404,225,21,11.67,1886,37.40,71.45,2022-02-01,00:57,2022-02-01,01:28,2022-02-01,00:50,2022-02-01,00:55,252.0
4,HV0003,B03404,25,17,2.63,700,11.43,22.24,2022-02-01,00:02,2022-02-01,00:14,2022-01-31,23:57,2022-02-01,00:01,283.0


In [3]:
df.columns

Index(['IdApp', 'IdProveedor', 'IdZonaOrigen', 'IdZonaDestino',
       'DistanciaViaje', 'trip_time', 'TarifaPasajero', 'CostoTotal',
       'FechaRecogida', 'HoraRecogida', 'FechaLlegada', 'HoraLlegada',
       'FechaSolicitada', 'HoraSolicitada', 'FechaAtendida', 'HoraAtendida',
       'DuracionAtencion'],
      dtype='object')

In [4]:
df.describe()

,IdZonaOrigen,IdZonaDestino,DistanciaViaje,trip_time,TarifaPasajero,CostoTotal,DuracionAtencion
count,2.845525e+06,2.845525e+06,2.845525e+06,2.845525e+06,2.845525e+06,2.845525e+06,2.845525e+06
mean,1.397041e+02,1.431089e+02,4.491413e+00,1.093993e+03,2.083887e+01,4.264973e+01,2.254768e+02
std,7.555066e+01,7.770159e+01,5.165545e+00,7.787493e+02,1.591658e+01,3.292686e+01,1.607226e+02
min,2.000000e+00,1.000000e+00,0.000000e+00,5.000000e+00,-1.334100e+02,-7.459000e+01,0.000000e+00
25%,7.600000e+01,7.600000e+01,1.480000e+00,5.630000e+02,1.071000e+01,2.144000e+01,1.150000e+02
50%,1.410000e+02,1.440000e+02,2.720000e+00,8.920000e+02,1.675000e+01,3.332000e+01,1.910000e+02
75%,2.120000e+02,2.200000e+02,5.500000e+00,1.398000e+03,2.554000e+01,5.248000e+01,2.980000e+02
max,2.650000e+02,2.650000e+02,4.967100e+02,9.858900e+04,1.452100e+03,2.743130e+03,1.158600e+04


In [24]:

# Filtrar solo las columnas numéricas
df_numeric = df.select_dtypes(include=['number'])

# Calcula el rango intercuartílico (IQR) para cada columna numérica
Q1 = df_numeric.quantile(0.25)
Q3 = df_numeric.quantile(0.75)
IQR = Q3 - Q1

# Define el límite inferior y superior para filtrar los outliers
limite_inferior = Q1 - 1.5 * IQR
limite_superior = Q3 + 1.5 * IQR

# Filtrar los outliers en las columnas numéricas
outliers = df_numeric[(df_numeric < limite_inferior) | (df_numeric > limite_superior)].any(axis=1)

# Filtrar los registros originales que no son outliers
df_no_outliers = df[~outliers]

# Identificar los registros que son outliers
outliers_data = df[outliers]

In [23]:
outliers_data

,IdApp,IdProveedor,IdZonaOrigen,IdZonaDestino,DistanciaViaje,trip_time,TarifaPasajero,CostoTotal,FechaRecogida,HoraRecogida,FechaLlegada,HoraLlegada,FechaSolicitada,HoraSolicitada,FechaAtendida,HoraAtendida,DuracionAtencion
3,HV0003,B03404,225,21,11.67,1886,37.40,71.45,2022-02-01,00:57,2022-02-01,01:28,2022-02-01,00:50,2022-02-01,00:55,252.0
16,HV0003,B03404,132,265,25.98,1940,51.51,116.82,2022-02-01,00:37,2022-02-01,01:09,2022-02-01,00:30,2022-02-01,00:37,423.0
28,HV0003,B03404,85,132,12.30,2063,40.25,57.82,2022-02-01,00:18,2022-02-01,00:53,2022-02-01,00:15,2022-02-01,00:17,110.0
76,HV0003,B03404,132,234,17.81,1602,41.89,102.07,2022-02-01,00:36,2022-02-01,01:02,2022-02-01,00:19,2022-02-01,00:36,988.0
89,HV0003,B03404,182,48,12.10,1172,30.11,60.47,2022-02-01,00:31,2022-02-01,00:50,2022-02-01,00:26,2022-02-01,00:30,234.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4004753,HV0003,B03404,129,13,12.44,3295,59.42,138.63,2022-02-08,07:49,2022-02-08,08:44,2022-02-08,07:45,2022-02-08,07:49,223.0
4004764,HV0003,B03404,17,36,2.36,1011,22.23,37.18,2022-02-08,07:30,2022-02-08,07:47,2022-02-08,07:19,2022-02-08,07:29,641.0
4004765,HV0003,B03404,36,237,9.06,3695,55.76,121.86,2022-02-08,07:52,2022-02-08,08:53,2022-02-08,07:46,2022-02-08,07:51,352.0
4004772,HV0003,B03404,215,26,13.92,5401,59.25,127.15,2022-02-08,07:49,2022-02-08,09:19,2022-02-08,07:41,2022-02-08,07:49,432.0


In [27]:
outliers_data.describe()

,IdZonaOrigen,IdZonaDestino,DistanciaViaje,trip_time,TarifaPasajero,CostoTotal,DuracionAtencion
count,369830.000000,369830.000000,369830.000000,369830.000000,369830.000000,369830.000000,369830.000000
mean,138.679001,151.268824,12.994775,2278.988827,46.153952,97.246415,359.500314
std,71.988377,83.069193,9.043786,1170.590666,27.566644,54.153673,276.142603
min,2.000000,1.000000,0.000000,46.000000,-133.410000,-74.590000,2.000000
25%,79.000000,80.000000,6.790000,1532.000000,32.190000,69.110000,150.000000
50%,138.000000,142.000000,12.660000,2156.000000,42.610000,91.590000,276.000000
75%,200.000000,232.000000,17.200000,2894.000000,54.630000,116.180000,579.000000
max,265.000000,265.000000,496.710000,98589.000000,1452.100000,2743.130000,11586.000000


In [25]:
df_no_outliers

,IdApp,IdProveedor,IdZonaOrigen,IdZonaDestino,DistanciaViaje,trip_time,TarifaPasajero,CostoTotal,FechaRecogida,HoraRecogida,FechaLlegada,HoraLlegada,FechaSolicitada,HoraSolicitada,FechaAtendida,HoraAtendida,DuracionAtencion
0,HV0003,B03404,17,225,1.89,611,9.85,19.18,2022-02-01,00:02,2022-02-01,00:13,2022-01-31,23:58,2022-02-01,00:02,258.0
1,HV0003,B03404,37,37,1.52,493,9.08,16.92,2022-02-01,00:20,2022-02-01,00:28,2022-02-01,00:16,2022-02-01,00:19,187.0
2,HV0003,B03404,37,37,1.50,456,11.75,19.57,2022-02-01,00:39,2022-02-01,00:46,2022-02-01,00:36,2022-02-01,00:38,145.0
4,HV0003,B03404,25,17,2.63,700,11.43,22.24,2022-02-01,00:02,2022-02-01,00:14,2022-01-31,23:57,2022-02-01,00:01,283.0
5,HV0003,B03404,17,225,1.98,871,16.63,31.78,2022-02-01,00:27,2022-02-01,00:41,2022-02-01,00:21,2022-02-01,00:25,239.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4004808,HV0003,B03404,260,226,1.55,994,6.18,18.37,2022-02-08,07:47,2022-02-08,08:04,2022-02-08,07:44,2022-02-08,07:45,49.0
4004809,HV0003,B03404,243,262,6.36,2591,38.12,80.88,2022-02-08,07:38,2022-02-08,08:21,2022-02-08,07:35,2022-02-08,07:38,171.0
4004812,HV0003,B03404,89,14,6.55,1833,25.72,51.75,2022-02-08,07:05,2022-02-08,07:36,2022-02-08,07:00,2022-02-08,07:04,211.0
4004813,HV0003,B03404,14,14,1.02,526,9.31,17.33,2022-02-08,07:41,2022-02-08,07:49,2022-02-08,07:37,2022-02-08,07:39,117.0


In [28]:
df_no_outliers.describe()

,IdZonaOrigen,IdZonaDestino,DistanciaViaje,trip_time,TarifaPasajero,CostoTotal,DuracionAtencion
count,2.475695e+06,2.475695e+06,2.475695e+06,2.475695e+06,2.475695e+06,2.475695e+06,2.475695e+06
mean,1.398572e+02,1.418900e+02,3.221144e+00,9.169728e+02,1.705720e+01,3.449384e+01,2.054558e+02
std,7.606731e+01,7.679321e+01,2.456744e+00,5.012409e+02,8.223614e+00,1.721186e+01,1.233502e+02
min,2.000000e+00,1.000000e+00,0.000000e+00,5.000000e+00,-1.152000e+01,-8.881784e-16,0.000000e+00
25%,7.500000e+01,7.600000e+01,1.390000e+00,5.310000e+02,1.013000e+01,2.037000e+01,1.120000e+02
50%,1.420000e+02,1.440000e+02,2.390000e+00,8.130000e+02,1.529000e+01,3.034000e+01,1.840000e+02
75%,2.130000e+02,2.160000e+02,4.350000e+00,1.206000e+03,2.216000e+01,4.476000e+01,2.810000e+02
max,2.650000e+02,2.650000e+02,1.153000e+01,2.650000e+03,4.778000e+01,9.904000e+01,5.720000e+02


In [22]:
data.head()

,IdProveedor,IdProveedor.1,IdZonaOrigen,IdZonaDestino,DistanciaViaje,trip_time,TarifaPasajero,CostoTotal,FechaRecogida,HoraRecogida,FechaLlegada,HoraLlegada,FechaSolicitada,HoraSolicitada,FechaAtendida,HoraAtendida,DuracionAtencion
0,HV0003,B03404,170,161,1.18,664,24.90,53.64,2022-01-01,00:07,2022-01-01,00:18,2022-01-01,00:05,2022-01-01,00:05,9.0
1,HV0003,B03404,237,161,0.82,460,11.97,28.46,2022-01-01,00:22,2022-01-01,00:30,2022-01-01,00:19,2022-01-01,00:22,161.0
2,HV0003,B03404,237,161,1.18,595,29.82,59.41,2022-01-01,00:57,2022-01-01,01:07,2022-01-01,00:43,2022-01-01,00:57,824.0
3,HV0003,B03404,262,229,1.65,303,7.91,17.90,2022-01-01,00:18,2022-01-01,00:23,2022-01-01,00:15,2022-01-01,00:17,92.0
4,HV0003,B03404,229,141,1.65,461,9.44,20.75,2022-01-01,00:28,2022-01-01,00:35,2022-01-01,00:25,2022-01-01,00:26,16.0


In [23]:
data.shape

(2665068, 17)

In [ ]:
#Calcular el tamaño de muestra proporcional para cada proveedor
proporciones = data['IdProveedor'].value_counts(normalize=True) * 0.12  # Proporción deseada
tamaño_muestra_por_proveedor = (proporciones * len(data)).astype(int)

# Lista para almacenar las muestras aleatorias estratificadas
muestras_estratificadas = []

# Iterar sobre cada proveedor único
for proveedor in data['IdProveedor'].unique():
    # Seleccionar aleatoriamente el tamaño de muestra proporcional para el proveedor actual
    muestra_proveedor = data[data['IdProveedor'] == proveedor].sample(tamaño_muestra_por_proveedor[proveedor])
    # Agregar las muestras seleccionadas al DataFrame de muestras estratificadas
    muestras_estratificadas.append(muestra_proveedor)

# Concatenar las muestras estratificadas en un solo DataFrame
muestras_estratificadas = pd.concat(muestras_estratificadas)

In [ ]:
muestras_estratificadas.shape

In [ ]:
muestras_estratificadas.IdProveedor.value_counts()

In [ ]:
muestras_estratificadas.head()

In [ ]:
muestras_estratificadas.dtypes

In [ ]:
# Conectar a la base de datos PostgreSQL
conn = psycopg2.connect(
    dbname="nycdb",
    user="postgres",
    password="admin",
    host="localhost"
)

In [ ]:
# Crear un cursor para ejecutar comandos SQL
cur = conn.cursor()

In [ ]:
# Crear la tabla en la base de datos
cur.execute("""
    CREATE TABLE service_yellow (
        Id SERIAL PRIMARY KEY,
        IdProveedor INTEGER,
        TotalPasajeros INTEGER,
        DistanciaViaje FLOAT,
        IdTipoTarifa FLOAT,
        IdZonaOrigen INTEGER,
        IdZonaDestino INTEGER,
        IdTipoPago INTEGER,
        CostoTotal FLOAT,
        FechaRecogida DATE,
        HoraRecogida INTEGER,
        DuracionViaje INTEGER
    )
""")

# Guardar los cambios en la base de datos
conn.commit()


# Convertir los valores del DataFrame a listas de Python
registros = muestras_estratificadas.values.tolist()

# Insertar los registros en la tabla
cur.executemany("""
    INSERT INTO service_yellow (
        IdProveedor, TotalPasajeros, DistanciaViaje, IdTipoTarifa, IdZonaOrigen, IdZonaDestino, IdTipoPago, CostoTotal, FechaRecogida, HoraRecogida, DuracionViaje) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
""", registros)

# Guardar los cambios en la base de datos
conn.commit()

# Cerrar la conexión con la base de datos
cur.close()
conn.close()

print("Registros migrados exitosamente a PostgreSQL.")